<a href="https://colab.research.google.com/github/tukamilano/anagram/blob/main/anagram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install fugashi unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.9/694.9 kB 27.1 MB/s eta 0:00:00
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=a59cad31aefe2d3ada2952d861ce4dbf7de7573e739c44c3cf021089e39c9143
  Stored in directory: /root/.cache/pip/wheels/5e/1f/0f/4d43887e5476d956fae828ee9b6687becd5544d68b51ed633d
Successfully built unidic-lite


In [2]:
import requests

# Hugging Face上のファイルURL
url = "https://huggingface.co/milano0017/hiragana_KenLM/resolve/main/model.bin"

# 保存先のファイル名
destination_file_name = "model.bin"

# ファイルをダウンロード
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(destination_file_name, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f"'{destination_file_name}' をダウンロードしました。")
else:
    print(f"エラー: ダウンロードに失敗しました (status code: {response.status_code})")


'model.bin' をダウンロードしました。


In [3]:
import os

# Clone KenLM if not already cloned
if not os.path.exists("kenlm"):
    !git clone https://github.com/kpu/kenlm.git
    print("KenLM repository cloned.")
else:
    print("KenLM repository already exists.")

# Build KenLM
# Change to kenlm directory
%cd kenlm

# Create build directory and navigate into it
!mkdir -p build
%cd build

# Configure and build KenLM
!cmake ..
!make -j $(nproc)

# Navigate back to the original content directory
%cd /content

print("KenLM build process complete. You can now try to run the lmplz command.")

Cloning into 'kenlm'...
remote: Enumerating objects: 14185, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 14185 (delta 117), reused 81 (delta 81), pack-reused 14009 (from 2)
Receiving objects: 100% (14185/14185), 5.91 MiB | 8.24 MiB/s, done.
Resolving deltas: 100% (8059/8059), done.
KenLM repository cloned.
/content/kenlm
/content/kenlm/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
CMake Warning (dev) at CMakeList

In [4]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

     / 553.6 kB 2.2 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp312-cp312-linux_x86_64.whl size=3188041 sha256=0e15849084a318a9af054babcc1462ba8848ba09ddff2d207ee1799fb32c1eea
  Stored in directory: /tmp/pip-ephem-wheel-cache-me0vj7ue/wheels/92/c8/12/56d187154e078f0eaa74d059017fc1afe1c4d91fbce02ce8d9
Successfully built kenlm


In [5]:
import kenlm

model = kenlm.Model("model.bin")

In [8]:
import fugashi

# ==== 初期化 ====
print("形態素解析器を初期化中...")
tagger = fugashi.Tagger()

形態素解析器を初期化中...


In [19]:
import random
import math
from itertools import combinations

# ===============
# ひらがなセット
# ===============
jp = list("あいうえおかきくけこさしすせそたちつてとなにぬねのはひふへほまみむめもやゆよらりるれろわをん")

# ===============================
# Perplexityを計算する関数
# ===============================
def calculate_perplexity(sentence, morph_num_weight=100):
    """
    KenLM を使って
    与えられた文の Perplexity（自然さ）を算出する。
    sentence: もしくは str
    """
    sentence = ''.join(sentence)
    processed_sentence_list = [word.surface for word in tagger(sentence)]
    processed_sentence = ' '.join(processed_sentence_list)
    ppl = model.perplexity(processed_sentence)
    #形態素の数にペナルティを与える
    ppl += morph_num_weight * len(processed_sentence_list)
    return ppl

In [25]:
def improve_by_4_opt(sentence, step=1000000):
    v = calculate_perplexity(sentence)
    calculate_num = 0
    improved = True

    while improved:
        improved = False
        indices = list(range(len(sentence)))
        base = list(combinations(indices, 4))
        random.shuffle(base)  # 組の順序をランダム化

        for four_var in base:
            a, b, c, d = four_var
            new_sentence = sentence[:a] + sentence[c:d] + sentence[b:c] + sentence[a:b] + sentence[d:]
            new_v = calculate_perplexity(new_sentence)
            calculate_num += 1

            if new_v < v:
                v = new_v
                sentence = new_sentence
                improved = True
                print(f"Improved! step={calculate_num}, new_ppl={v:.3f}")
                break

            if calculate_num >= step:
                print(f"Reached step limit ({step}). Stop.")
                return sentence

    print("No more improvement.")
    return sentence

def improve_by_4_opt_sa(sentence, step=1_000_000, T0=1000.0, Tmin=0.1, alpha=0.99999):
    v = calculate_perplexity(sentence)
    best_sentence, best_v = sentence, v
    T = T0

    for step_num in range(step):
        # ランダムに4点選択
        indices = sorted(random.sample(range(len(sentence)), 4))
        a, b, c, d = indices
        new_sentence = sentence[:a] + sentence[c:d] + sentence[b:c] + sentence[a:b] + sentence[d:]
        new_v = calculate_perplexity(new_sentence)
        delta = new_v - v

        # 改善 or 確率的悪化許容
        if delta < 0 or random.random() < math.exp(-delta / T):
            sentence, v = new_sentence, new_v
            if v < best_v:
                best_sentence, best_v = sentence, v
                print(f"Step {step_num}: Improved! new_ppl={v:.3f}")

        # 温度を減衰
        T = max(T * alpha, Tmin)

        # ログ出力（確認用）
        if step_num % 1000 == 0:
            print(f"Step {step_num}: T={T:.3f}, ppl={v:.3f}")

    print(f"Finished all {step} steps.")
    print(f"Best perplexity: {best_v:.3f}")
    return best_sentence


# ========================
# 実行部分
# ========================
def main():
    sentence = jp[:]  # コピーしておく
    random.shuffle(sentence)
    print("Initial sentence:", "".join(sentence))
    improved_sentence = improve_by_4_opt_sa(sentence)
    print("Final sentence:", "".join(improved_sentence))

if __name__ == "__main__":
    main()


Initial sentence: そもいすやむまんとかなはりみこてねせひえさゆきつれのへあおらちわるろめくけほたうによぬをしふ
Step 0: T=999.990, ppl=11309.297
Step 1: Improved! new_ppl=7915.956
Step 2: Improved! new_ppl=7050.447
Step 3: Improved! new_ppl=5887.241
Step 11: Improved! new_ppl=5312.920
Step 12: Improved! new_ppl=5099.873
Step 14: Improved! new_ppl=5012.925
Step 25: Improved! new_ppl=4879.586
Step 42: Improved! new_ppl=4713.044
Step 50: Improved! new_ppl=4671.882
Step 110: Improved! new_ppl=4654.203
Step 530: Improved! new_ppl=4611.816
Step 532: Improved! new_ppl=4606.545
Step 659: Improved! new_ppl=4429.701
Step 1000: T=990.040, ppl=8390.805
Step 2000: T=980.189, ppl=8860.690
Step 2318: Improved! new_ppl=4413.034
Step 2324: Improved! new_ppl=4412.093
Step 3000: T=970.436, ppl=5979.616
Step 4000: T=960.780, ppl=6453.860
Step 5000: T=951.220, ppl=5990.702
Step 6000: T=941.755, ppl=7729.921
Step 6602: Improved! new_ppl=4283.173
Step 7000: T=932.384, ppl=5094.019
Step 8000: T=923.107, ppl=5824.301
Step 9000: T=913.922, ppl=5891.457
Step

In [26]:
text = "ひろみはことさらによせかけたおやもそのてえまほしくなりいわゆるへんうちぬれあねむすめをきつふ"
processed_sentence = ' '.join(word.surface for word in tagger(text))
print(processed_sentence)

ひろみ は ことさら に よせかけ た おや も その て え ま ほしく なり いわゆる へん うち ぬれ あねむすめ を きつ ふ
